In [1]:
%%configure -f
{
    "jars":["/home/lorenzo/EdgeCloudApproximate/libraries/magellan-1.0.5-s_2.11.jar"],
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,com.esri.geometry:esri-geometry-api:1.2.1,commons-io:commons-io:2.6",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11",
        
        "spark.dynamicAllocation.enabled": false
    }
}


In [2]:
/* in the above cell, you need to load the magellan-1.0.5-s_2.11.jar file as a local file
using "jars": */

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,spark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
/**
 * @Description: georeferenced data online stratified sampling
 * @author: Isam Al Jawarneh
 * @date: 20/Jan/2023
 */

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
// sc.getConf.getAll.foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
// sc.getConf.get("spark.dynamicAllocation.enabled")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}

import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{
  DoubleType,
  StringType,
  StructField,
  StructType
}
import org.apache.spark.sql.streaming._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions.{collect_list, collect_set}
import org.apache.spark.sql.SQLContext
import org.apache.log4j.{Level, Logger}
import scala.collection.mutable
import scala.concurrent.duration.Duration
import java.io.{BufferedWriter, FileWriter}
import org.apache.commons.io.FileUtils
import java.io.File
import scala.collection.mutable.ListBuffer
import java.time.Instant
// import org.apache.spark.util.CollectionAccumulator
import org.apache.spark.sql.DataFrame
import org.apache.kafka.clients.consumer.ConsumerRecord
import org.apache.kafka.common.serialization.StringDeserializer

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import util.control.Breaks._
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.types._
import org.apache.spark.rdd.RDD
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.SQLImplicits
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.ForeachWriter
import magellan._
import magellan.index.ZOrderCurve
import magellan.{Point, Polygon}
import org.apache.spark.sql.magellan.dsl.expressions._
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.streaming.OutputMode
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}
import org.apache.spark.s

In [7]:
// use the folloing link to create topic in Kafka if the topic is not there already!

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

%%bash
#create topic 'spatial' with 4 partitions
export KafkaZookeepers="localhost:2181"

/home/kafka/kafka/bin/kafka-topics.sh --create --replication-factor 1 --partitions 4 --topic spatial1 --zookeeper $KafkaZookeepers

In [8]:
// list the available Kafka topics

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

%%bash
#list topics to check
export KafkaZookeepers="localhost:2181"
/home/kafka/kafka/bin/kafka-topics.sh --list --zookeeper $KafkaZookeepers

In [ ]:
// create Kafka stream

In [10]:

val stream = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("startingOffsets", "earliest").option("subscribe", "spatial1").load() // .option("maxOffsetsPerTrigger",2).option("startingOffsets", "earliest")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

stream: org.apache.spark.sql.DataFrame = [key: binary, value: binary ... 5 more fields]


In [11]:
val schemaShenzhen = StructType(Array(
    StructField("id", StringType, false),
    StructField("lat", DoubleType, false),
    StructField("lon", DoubleType, false),
    StructField("time", StringType, false),
    StructField("speed", DoubleType, false)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

schemaShenzhen: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,false), StructField(lat,DoubleType,false), StructField(lon,DoubleType,false), StructField(time,StringType,false), StructField(speed,DoubleType,false))


In [12]:
// parameters

val sampling_fraction = 0.5
val precision = 30
val error_bound = 0.1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sampling_fraction: Double = 0.5
precision: Int = 30
error_bound: Double = 0.1


In [13]:
// the following user defined function is required to convert ZOrderCurve to geohash base32

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
val geohashUDF = udf{(curve: Seq[ZOrderCurve]) => curve.map(_.toBase32())}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

geohashUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(ArrayType(org.apache.spark.sql.types.ZOrderCurveUDT@2a820cc3,true))))


In [15]:
// ramp - up: before running the following cell, generate some data from the CSV file then ingest by Kafka topic
// created previously

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
// note that 'precision' in the following code is a configurable parameter and it is the geohash precision 
// precision 30 means 6 characters , precision 25 means 5 characters (multiples of 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
val transformationStream1 = stream.selectExpr("CAST(value AS STRING)").as[(String)].select(from_json($"value", schemaShenzhen).as("data")).select("data.*")

val ridesGeohashed = transformationStream1.withColumn("point", point($"lat",$"lon")).withColumn("index", $"point" index  precision).withColumn("geohashArray", geohashUDF($"index.curve")).select( $"point",$"geohashArray",$"speed")
val dataStream1 = ridesGeohashed.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }
val transformationStream = dataStream1.select("point", "speed", "geohash")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

transformationStream1: org.apache.spark.sql.DataFrame = [id: string, lat: double ... 3 more fields]
ridesGeohashed: org.apache.spark.sql.DataFrame = [point: point, geohashArray: array<string> ... 1 more field]
dataStream1: org.apache.spark.sql.DataFrame = [point: point, geohashArray: array<string> ... 2 more fields]
transformationStream: org.apache.spark.sql.DataFrame = [point: point, speed: double ... 1 more field]


In [18]:
/* random sampling from each stratum individually,
map consists of strata and percentage for each stratum
*/
 def getSamplingRate1(map: Map[String, Double], defaultValue: Double) = udf{
  (geohash: String, rnd: Double) =>
      rnd < map.getOrElse(geohash.asInstanceOf[String], 0.0)
}

// spatial stratified sampling
def spatialSampleBy(neigh_geohashed_df:DataFrame, points_geohashed_df:DataFrame, samplingRatio: Double): DataFrame = {
    val geoSeq: Seq[String] = neigh_geohashed_df.select("geohash").distinct.rdd.map(r => r(0).asInstanceOf[String]).collect()
    val map = Map(geoSeq map { a => a -> samplingRatio }: _*)

        val tossAcoin = rand(7L)
    val getSamplingRate = udf { (geohash: Any, rnd: Double) =>
      rnd < map.getOrElse(geohash.asInstanceOf[String], 0.0)
    }
val samplepointDF =  points_geohashed_df.filter(getSamplingRate1(map, 0.0)($"geohash", tossAcoin))
    return samplepointDF}


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

getSamplingRate1: (map: Map[String,Double], defaultValue: Double)org.apache.spark.sql.expressions.UserDefinedFunction
spatialSampleBy: (neigh_geohashed_df: org.apache.spark.sql.DataFrame, points_geohashed_df: org.apache.spark.sql.DataFrame, samplingRatio: Double)org.apache.spark.sql.DataFrame


In [19]:
def geohashedNeighborhoods(geohashPrecision: Int, filePath: String): DataFrame = 

{

import spark.implicits._
/* preparing the neighborhoods table (static table) .... getting geohashes covering for every neighborhood and 
exploding it, so that each neighborhood has many geohashes */

// this will be executed only one time - batch mode 
val rawNeighborhoods = spark.sqlContext.read.format("magellan").option("type", "geojson").load(filePath).select($"polygon", $"metadata"("NAME").as("neighborhood")).cache()

val neighborhoods = rawNeighborhoods.withColumn("index", $"polygon" index geohashPrecision).select($"polygon", $"index", 
      $"neighborhood").cache()
    print(neighborhoods.count())

val zorderIndexedNeighborhoods = neighborhoods.withColumn("index", explode($"index")).select("polygon", "index.curve", "index.relation","neighborhood")
val geohashedNeighborhoods= neighborhoods.withColumn("geohashArray", geohashUDF($"index.curve"))

val explodedgeohashedNeighborhoods = geohashedNeighborhoods.explode("geohashArray", "geohash") { a: mutable.WrappedArray[String] => a }

//unit testing: explodedgeohashedNeighborhoods.show(10)


explodedgeohashedNeighborhoods

}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

geohashedNeighborhoods: (geohashPrecision: Int, filePath: String)org.apache.spark.sql.DataFrame


In [20]:
// read geoJSON file containing Neighborhoods

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
val lorenzoRepo = "/home/lorenzo/EdgeCloudApproximate"
val geohashedNeigboors = geohashedNeighborhoods(precision, "/home/lorenzo/EdgeCloudApproximate/data/china/neighborhood")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lorenzoRepo: String = /home/lorenzo/EdgeCloudApproximate
12geohashedNeigboors: org.apache.spark.sql.DataFrame = [polygon: polygon, index: array<struct<curve:zordercurve,relation:string>> ... 3 more fields]


In [52]:
/* careful here we perform online spatial sampling. this means sampling on-the-fly (OTF) 
as we are taking data directly from transformationStream, which by itself a transformation from a Kafka stream 
'stream'
*/
val samplepointDF_SSS = spatialSampleBy(geohashedNeigboors,transformationStream,sampling_fraction)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

samplepointDF_SSS: org.apache.spark.sql.DataFrame = [point: point, speed: double ... 1 more field]


In [53]:
val samplingStatisticsDF  = samplepointDF_SSS.groupBy($"geohash").agg(avg($"speed").as("per_strat_mean"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

samplingStatisticsDF: org.apache.spark.sql.DataFrame = [geohash: string, per_strat_mean: double]


In [54]:
/*
thereafter, we output data to a local in-memory sink
to be able to perform queries locally over already-aggregated stream data.
so, this way we are writing only sumamries in-memory, which is more effecient
*/
val points_new = samplingStatisticsDF.writeStream.queryName("queryTable").format("memory").outputMode("complete").start()//outputMode("append")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

points_new: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3dc3d4c3


In [55]:
// check wether the stream is active
points_new.isActive

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res70: Boolean = true


In [58]:
val population = spark.sql("select * from queryTable")
          
population.createOrReplaceTempView("updates")
val tau_hat_str = spark.sql("select per_strat_mean from updates").head().getDouble(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

population: org.apache.spark.sql.DataFrame = [geohash: string, per_strat_mean: double]
tau_hat_str: Double = 11.4


In [59]:
population.show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|geohash|    per_strat_mean|
+-------+------------------+
| ws11p7|              11.4|
| ws10tn| 22.61904761904762|
| ws1171|11.583333333333334|
| ws11pk| 6.416666666666667|
| ws0crg|              27.0|
| ws10un|              41.0|
| ws10u4| 18.77777777777778|
| ws1078| 9.497098646034816|
| ws10fz|              10.0|
| ws10fh|              13.2|
| ws106e|16.044025157232703|
| ws10s6|17.136363636363637|
| ws1111| 22.90909090909091|
| ws1098| 8.857142857142858|
| ws10mg|14.394160583941606|
| ws0crz|              13.5|
| ws10es|15.321428571428571|
| ws10mq| 9.264367816091953|
| ws10ve|               9.0|
| wecpbj| 9.212389380530974|
| ws109r|              12.0|
| ws10kd|20.890845070422536|
| ws0brz|37.093023255813954|
| ws0cw6|30.333333333333332|
| ws10hz| 7.662904439428141|
| ws112y|              17.0|
| ws0brk|20.414285714285715|
| ws0cq6|16.166666666666668|
| ws0bzs|              50.2|
| ws1142| 6.333333333333333|
+-------+------------------+
only showing t

In [28]:
// points.stop

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
// run the following cell if you only want to delete or mark for delection a Kafka topic

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

%%bash
/home/kafka/kafka/bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic spatial1
#sudo systemctl restart kafka
#sudo systemctl status kafka

In [51]:
points_new.stop

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…